In [3]:
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import geojson

def merge_provinces(geojson_file, province_names):
    with open(geojson_file, 'r') as f:
        data = geojson.load(f)

    provinces_to_merge = [feature for feature in data['features'] if feature['properties']['NAME_1'] in province_names]

    shapes = [shape(feature['geometry']) for feature in provinces_to_merge]

    merged = unary_union(shapes)

    merged_geojson = geojson.Feature(geometry=mapping(merged), properties={"NAME_1": "Merged Area"})

    return merged_geojson

def write_to_geojson(merged_provinces, output_file):
    feature_collection = geojson.FeatureCollection(merged_provinces)

    with open(output_file, 'w') as f:
        geojson.dump(feature_collection, f)


In [4]:
province_groups = [
    ("MNF-W", ["Al-Anbar"]),
    ("MND-C", ["Karbala", "Wasit", "Al-Qadisiyah", "Al-Najaf", "Babil"]),
    ("MND-SE", ["Al-Basrah", "Maysan", "Al-Muthannia", "Dhi-Qar"]),
    ("MND-BAGHDAD", ["Baghdad"]),
    ("MND-N", ["At-Ta'mim", "Sala ad-Din", "Diyala", "As-Sulaymaniyah", "Ninawa", "Dihok", "Arbil"]),
]

merged_provinces = []

for new_name, province_names in province_groups:
    merged_province = merge_provinces("iraq_provinces.geojson", province_names)
    merged_province["properties"]["NAME_1"] = new_name
    merged_provinces.append(merged_province)

write_to_geojson(merged_provinces, "merged_provinces.geojson")
